In [2]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup, Comment
import urllib

In [3]:

link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
f = urllib.request.urlopen(link)
info_raw=f.read()
#retriving HTml Code

In [4]:
soup = BeautifulSoup(info_raw,'lxml')

In [5]:
table = soup.table #Table

In [6]:
df = pd.read_html(str(table))[0]
df.head() #reading data 

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
df = df[df['Borough'] != 'Not assigned']
df.head() #removing rows in which Borough is not assigned

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:

df['Neighborhood']=df['Neighborhood'].str.replace('/',',')
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
df=df[~df['Neighborhood'].str.contains('Not assigned')] #rechecking the other column 

In [12]:
df.shape #shape of the table rows and columns 

(103, 3)

In [19]:
!pip install pgeocode
import pgeocode
nomi = pgeocode.Nominatim('CA')

In [27]:
df

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.7545,-79.3300
M4A,North York,Victoria Village,43.7276,-79.3148
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
M3B,North York,Don Mills,43.7450,-79.3590
M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094


In [29]:
!pip install folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import requests # library to handle requests

     |████████████████████████████████| 102kB 6.5MB/s ta 0:00:011


In [30]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [31]:

# Remove rows that does not ontains coordinates informaiton
df = df[df['Longitude'].notnull()]

In [32]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index()
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


In [33]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto